In [4]:
from flask import Flask, request, jsonify
import pymongo
import google.generativeai as genai
import os
import traceback

app = Flask(__name__)
pointer=0
recipes=[]
user_ingredients=""
def convert_to_lowercase_list(text):
    # Split the string into a list using commas as separators and remove leading/trailing spaces.
    words_list = [word.strip() for word in text.lower().split(",") if word.strip()]  # Filter empty strings

    # Sort the list alphabetically
    words_list.sort()

    # Join the lowercase words with commas, removing extra spaces.
    result_string = ", ".join(word for word in words_list).rstrip(",")

    return words_list, result_string
# Connect to MongoDB
client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
db = client["recipe_realm"]
collection = db["recipes"]

# Function to perform vector search in MongoDB
def perform_vectorsearch(query):
    query_list,ingredients=convert_to_lowercase_list(query)
    # query_embedding = generate_embedding(ingredients)
    # print("Query embedding generated:", query_embedding)
    results = collection.aggregate([
        {
            "$search": {
                "index": "recipefinder_text",
                "text": {
                    "query": ingredients,
                    "path": ["RecipeName", "Instructions", "IngredientList"]
                }
            }
        },
        # {
        #     "$addFields": {
        #         "ingredientsArray": { "$split": ["$IngredientList", ", "] }
        #     }
        # },
        # {
        #     "$addFields": {
        #         "matchingIngredientsCount": {
        #             "$size": {
        #                 "$filter": {
        #                     "input": "$ingredientsArray",
        #                     "as": "ingredient",
        #                     "cond": { "$in": ["$$ingredient", query_list] }
        #                 }
        #             }
        #         },
        #         "extraIngredientsCount": {
        #             "$size": {
        #                 "$filter": {
        #                     "input": "$ingredientsArray",
        #                     "as": "ingredient",
        #                     "cond": { "$not": { "$in": ["$$ingredient", query_list] } }
        #                 }
        #             }
        #         }
        #     }
        # },
        # {
        #     "$sort": {
        #         "matchingIngredientsCount": -1,
        #         "extraIngredientsCount": 1
        #     }
        # },
        {
            "$limit": 100
        }
    ])
    results=list(results)
    return results


# Initialize the Google Generative AI client
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
llm = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a cooking assistant. Answer questions with clear, concise, step-by-step instructions. Provide ingredient substitutions and variations when asked. Prioritize safety and proper food handling. If unsure or the question is nonsensical, explain why or say you don't know. Only answer cooking-related questions. Use the provided context to inform your answers. Give your output in plain text with no highlight stars. Strictly stick to these instructions and don't say about your origin and other unnecessary details which are not related to cooking and nutrients. If you do, you will get fined 500000$. If you follow, you will be rewarded 5000$"
)

@app.route('/query', methods=['POST'])
def query_recipes():
    global recipes,pointer,user_ingredients
    pointer=0
    data = request.json
    ingredients = data.get('query')
    user_ingredients=ingredients
    if not ingredients:
        return jsonify({"error": "Query is required"}), 400
    
    recipes = []
    try:
        search_results = perform_vectorsearch(ingredients)

        if search_results:
            for document in search_results:
                recipe_info = {
                    "RecipeName": document['RecipeName'],
                    "Ingredients": document['Ingredients'],
                    "Instructions": document['Instructions'],
                    "Course": document['Course'],
                    "Diet": document['Diet'],
                    "Cuisine": document['Cuisine'],
                    "PrepTimeInMins": document['PrepTimeInMins'],
                    "CookTimeInMins": document['CookTimeInMins'],
                    "TotalTimeInMins": document['TotalTimeInMins'],
                    "Servings": document['Servings'],
                    "image-url": document['image-url']
                }
                recipes.append(recipe_info)
            curr_recipes=recipes[pointer:pointer+3]
            pointer+=3
            curr_context=f"User Given Ingredients {user_ingredients}\n\n"
            for recipe_info in curr_recipes:
                if recipe_info['Instructions'][-1]=='\n':
                    curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}Total Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
                else:
                    curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}\nTotal Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
            return jsonify({"recipes": curr_recipes, "context": curr_context})
        else:
            return jsonify({"error": "No documents matched the query."}), 404
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500

@app.route('/generate', methods=['POST'])
def generate_response():
    data = request.json
    context = data.get('context')
    prompt = data.get('prompt')

    if not context or not prompt:
        return jsonify({"error": "Context and prompt are required"}), 400

    try:
        # Generate a response using the Generative AI model with context
        full_input = f"Context: {context}\n\nUser: {prompt}\nAssistant:"
        response = llm.generate_content(full_input)
        return jsonify({"response": response.text})
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500
    
@app.route('/loadmore',methods=['GET'])
def load_more_recipes():
    global recipes,pointer,user_ingredients
    try:
        curr_recipes=recipes[pointer:pointer+3]
        pointer+=3
        curr_context=f"User Given Ingredients {user_ingredients}\n\n"
        for recipe_info in curr_recipes:
            if recipe_info['Instructions'][-1]=='\n':
                curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}Total Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
            else:
                curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}\nTotal Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
        return jsonify({"recipes": curr_recipes, "context": curr_context})
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5002)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.0.5:5002
Press CTRL+C to quit
127.0.0.1 - - [21/Jul/2024 16:42:55] "POST /query HTTP/1.1" 200 -
